In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import psutil

In [ ]:
memory = psutil.virtual_memory()
memory.total

In [ ]:
16759816192 / 1000 / 1000 / 1000

In [ ]:
import re
from glob import glob

from IPython.display import display

from itertools import product

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from scipy.interpolate import RegularGridInterpolator

from pymedphys.dcm import coords_and_dose_from_dcm
from pymedphys.gamma import gamma_percent_pass

In [ ]:
# x and y labels within this notebook are likely wrong

In [ ]:
lower_percent_dose_cutoff = 20

In [ ]:
gamma_base_options = {
    'dose_percent_threshold': 3,
    'distance_mm_threshold': 3,
    'interp_fraction': 10,
    'max_gamma': 1.1,
    'lower_percent_dose_cutoff': lower_percent_dose_cutoff,
    'skip_once_passed': True
}

In [ ]:
ids = [6283, 1871, 5062]
# tps = ['Ecl', 'Pin']
# options = ['', '-masked']

In [ ]:
def get_filename(an_id, a_tps, an_option):
    return 'RD.{0}_{1}{2}.dcm'.format(an_id, a_tps, an_option)


def get_filepath(an_id, a_tps, an_option):
    filename = get_filename(an_id, a_tps, an_option)
    return '/home/simon/Dropbox/Shared/MedPhysData/{}/{}'.format(an_id, filename)

In [ ]:
combos = np.concatenate([
    [((an_id, 'Pin', ''), (an_id, 'Ecl', ''),),
    ((an_id, 'Pin', ''), (an_id, 'Ecl', '-masked'),),
    ((an_id, 'Pin', '-masked'), (an_id, 'Ecl', '-masked'),)]
    for an_id in ids
])

combos

In [ ]:
def get_comparison_files(compare_array):
    evaluate = get_filepath(*compare_array[0])
    reference = get_filepath(*compare_array[1])
    
    return evaluate, reference


get_comparison_files(combos[0])

In [ ]:
columns=[
    'reference',
    'evaluation',
    'gamma_shell_local_gamma_no_mask_evaluation',
    'gamma_shell_local_gamma_mask_evaluation',
    'gamma_shell_no_mask_evaluation',
    'gamma_shell_with_mask_evaluation'
]

df = pd.DataFrame(
    columns=columns
)


row = pd.Series(
    index=columns,
    data=[None,]*6
)
# df.append(row, ignore_index=True)
row
row['reference'] = 'a'

row

df.append(row, ignore_index=True)

In [ ]:
options_map = {
    'gamma_shell_local_gamma_no_mask_evaluation': {
        'method': 'shell',
        'mask_evaluation': False,
        'local_gamma': True
    },
    'gamma_shell_local_gamma_mask_evaluation': {
        'method': 'shell',
        'mask_evaluation': True,
        'local_gamma': True
    },
    'gamma_shell_no_mask_evaluation': {
        'method': 'shell',
        'mask_evaluation': False,
        'local_gamma': False
    },
    'gamma_shell_with_mask_evaluation': {
        'method': 'shell',
        'mask_evaluation': True,
        'local_gamma': False
    }
}

def get_options(combo, column_name):
    filepath_compare = get_comparison_files(combo)
    
    return {
        **gamma_base_options, 
        **{
            'dcm_ref_filepath': filepath_compare[1],
            'dcm_eval_filepath': filepath_compare[0],
        },
        **options_map[column_name]
    }

In [ ]:
df = pd.DataFrame(
    columns=columns
)

for combo in combos:
    row = pd.Series(
        index=columns,
        data=[None,]*6
    )
    row['reference'] = get_filename(*combo[1])
    row['evaluation'] = get_filename(*combo[0])
    
    for column_name in options_map:
        gamma_options = get_options(combo, column_name)
        print('Calcing {}'.format(column_name))
        percent_pass = gamma_percent_pass(**gamma_options)
        print(' | DONE') 
        
        row[column_name] = percent_pass
        
        display(row)
        
    df = df.append(row, ignore_index=True)
    display(df)

In [ ]:
df